# skinny prediction and advisory dashboard

### `MyVineyard` (vineyard properties including `grape variety`, `soil type`, `rootstock type`, `...`,  and also variables that will affecct management intervention such as `canopy structure` or `vine width`); 

### `MyManagement` (management related variables such as `entitlement`, `water delivery service`, `...`)

In [ ]:
from __future__ import print_function
import os
import ipywidgets as ipyw
import matplotlib as mpl
import matplotlib.pyplot as plt
import IPython
import warnings
warnings.filterwarnings("ignore")
import mjknowling

In [ ]:
%matplotlib inline
# widgets
irrig_scen = ["low", "high"]
irrig_str = ipyw.Text("0.5")
str_d = {"irrigation": irrig_str}
plot_types = ["irrigationtimeseries", "irrigationcost", "harvestyield", "harvestrevenue",
              "laitimeseries", "costcontributions", "grossmargin"]
plot_str = ipyw.Text("0.5")
p_d = {"plot": plot_str}

def set_irrig(**kwargs):
    for k, v in kwargs.items():
        str_d[k].value = str(v)
        
def set_plot_type(**kwargs):
    for k, v in kwargs.items():
        p_d[k].value = str(v)
        
w = ipyw.interactive(set_irrig, irrigation=irrig_scen)
p = ipyw.interactive(set_plot_type, plot=plot_types)

def on_button_clicked(b):
    irrig = str_d['irrigation'].value
    print("evaluating {} irrigation scenario with respect to base scenario...".format(irrig))
    scens = mjknowling.run_scen(irrig)
    plot = p_d['plot'].value
    print("plotting {}".format(plot))
    mjknowling.plot_scen(scens, plot)
    
button = ipyw.Button(description="Get Some Science")
button.on_click(on_button_clicked)

In [ ]:
display(w)

In [ ]:
display(p)  # or just streams to reduce model calls?

In [ ]:
display(button)